In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

import numpy as np
from random import randint
from math import floor, ceil
import codecs

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace
from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta
from tqdm import tqdm

exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
day_chosen = hoje

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{day_chosen.strftime('%d_%m_%Y')}", compress=True)
cc.df.shape

In [ ]:
# ANTES
cc.df.groupby('evolucao')[['id_notifica']].count()

In [ ]:
cc.df['reinfeccao'] = cc.df['reinfeccao'].fillna(False)
cc.df.groupby(['reinfeccao'])[['id_notifica']].count()

In [ ]:
cc.df['reinfeccao_ids'] = cc.df['reinfeccao_ids'].fillna(-1)

In [ ]:
cc.df = cc.df.drop(columns=['exame_nome', 'acc_merge_notifica'])

In [ ]:
cc.df.columns

In [ ]:
notifica = Notifica()
notifica.load('notifica', compress=False)
notifica.df = notifica.df.drop_duplicates('id', keep='last')
notifica.df.shape

In [ ]:
updateable_columns = pd.read_csv(join(default_input, "updateable_columns.csv"))
updateable_columns = updateable_columns.loc[updateable_columns['updateable']==1]

updateable_columns_list = updateable_columns['column'].to_list()
updateable_columns

In [ ]:
for column in updateable_columns['column']:
    if (column not in cc.df.columns):
        print(f"Creating column {column}")
        cc.df[column] = None 

In [ ]:
# ATUALIZAÇÃO

notifica.df.set_index('id', inplace=True)
cc.df.set_index('id_notifica', inplace=True)

cc.df.update(notifica.df[updateable_columns_list])

In [ ]:
notifica.df.reset_index(inplace=True)
cc.df.reset_index(inplace=True)

In [ ]:
# DEPOIS
cc.df.groupby('evolucao')[['id_notifica']].count()

In [ ]:
# cc.df.groupby(['criterio_classificacao', 'resultado'])[['id_notifica']].count().rename(index=cc.tables['criterio_classificacao'].set_index('id')['criterio_classificacao'], level='criterio_classificacao').rename(index=cc.tables['resultado'].set_index('id')['resultado'], level='resultado')

In [ ]:
cc.fix_dtypes()

In [ ]:
cc.normalize()

In [ ]:
cc.hashes()

In [ ]:
cc.df.groupby(['reinfeccao'])[['id_notifica']].count()

In [ ]:
cc.df.dtypes

In [ ]:
cc.df.shape

In [ ]:
# # (ANTES)
# for i in update_columns:
#     print(f" {i} -- > {len(cc.df.loc[cc.df[i].isna()])}")

In [ ]:
# PREPARAÇÃO 

In [ ]:
# notifica.df.set_index('id', inplace=True)
# cc.df.set_index('id_notifica', inplace=True)

# cc.df.update(notifica.df[update_columns])

# notifica.df.reset_index(inplace=True)
# cc.df.reset_index(inplace=True)

In [ ]:
# # (DEPOIS)
# for i in update_columns:
#     print(f" {i} -- > {len(cc.df.loc[cc.df[i].isna()])}")

In [ ]:
cc.save(f"cc_{day_chosen.strftime('%d_%m_%Y')}-new", replace=True, compress=True)